In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import imageio.v3 as iio
from matplotlib.colors import Normalize

from metaparc.data.dataset_utils import PhysicsDataset

In [ ]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

    # add scalar values to the input fields
data_path = Path(r"C:\Users\zsa8rk\Coding\MetaPARC\data\datasets\object_periodic_flow_water/data")
    
for file in data_path.glob("**/*.hdf5"):
    print(file)
    with h5py.File(file, 'r+') as f:
        # for key, val in f.attrs.items():
        #     print(key, val)
        # f.visititems(print_attrs)


        # give correct name to the file
        f.attrs["dataset_name"] = "COMSOL_ObjectPeriodicFlowWater"

        # # get the Re and cyl_dia from the attrs
        # Re = f.attrs["Re"]
        # cyl_dia = f.attrs["cyl_dia"]

        # get the vel_in from the attrs
        vel_in = f.attrs["inlet_vel"]

        scalars = f["scalars"]

        if "inlet_vel" not in scalars.keys():
            vel_in_dset = scalars.create_dataset("inlet_vel", data=vel_in)
            vel_in_dset.attrs["sample_varying"] = False
            vel_in_dset.attrs["time_varying"] = False

        # # if "Re" not in scalars.keys():
        # #     Re_dset = scalars.create_dataset("Re", data=Re)
        #     Re_dset.attrs["sample_varying"] = False
        #     Re_dset.attrs["time_varying"] = False

        # if "cyl_dia" not in scalars.keys():
        #     cyl_dia_dset = scalars.create_dataset("cyl_dia", data=cyl_dia)
        #     cyl_dia_dset.attrs["sample_varying"] = False
        #     cyl_dia_dset.attrs["time_varying"] = False

In [ ]:
# Load HDF5 file
data_path = Path(r"C:\Users\zsa8rk\Coding\MetaPARC\data\datasets\object_periodic_flow_water\data\object_perio_flow_vel_1.5_2.hdf5")
# data_path = Path(r"C:\Users\zsa8rk\Coding\MetaPARC\data\datasets\cylinder_wall_flow\data\cylinder_wall_flow_Re_300.0_cyl_dia_0.3.hdf5")  # Adjust path as needed
with h5py.File(data_path, 'r') as f:
    vel = f["t1_fields"]['velocity'][:]  # Shape assumed to be (n_traj, time, height, width, 2)
    p = f["t0_fields"]['pressure'][:]

print(vel.shape)
print(p.shape)

In [ ]:
# Load Well Dataset
path = Path("C:/Users/zsa8rk/Coding/MetaPARC/data/datasets/shear_flow/data/train")
dataset = WellDataset(str(path))

metadata = dataset.metadata
print(metadata)

data = dataset[0]
for key, value in data.items():
    print(key, value.shape)


In [ ]:
time_step = 1

input_fields = data["input_fields"]

tracer = input_fields[time_step, ..., 0]
pressure = input_fields[time_step, ..., 1]
vel_x = input_fields[time_step, ..., 2]
vel_y = input_fields[time_step, ..., 3]

fig, axes = plt.subplots(2, 3, figsize=(15, 5))

In [ ]:
# Visualize velocity magnitude

# Get velocity magnitude for first trajectory
traj_idx = 0
vel_mag = np.linalg.norm(vel[traj_idx, :, :, :, :], axis=-1)  # Take first timestep

# get pressure for first trajectory
p = p[traj_idx, :, :, :]
time_idx = [10,50,100]
# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 5))

# Plot velocity magnitude for each timestep
for i, t in enumerate(time_idx):
    im = axes[0,i].imshow(vel_mag[t], cmap='viridis')
    plt.colorbar(im, ax=axes[0,i], label='Velocity Magnitude')
    
    # Add labels and title
    axes[0,i].set_xlabel('X')
    axes[0,i].set_ylabel('Y')
    axes[0,i].set_title(f'Velocity Magnitude Field (t={t})')

for i, t in enumerate(time_idx):
    im = axes[1,i].imshow(p[t], cmap='plasma')
    plt.colorbar(im, ax=axes[1,i], label='Pressure')
    
    # Add labels and title
    axes[1,i].set_xlabel('X')
    axes[1,i].set_ylabel('Y')
    axes[1,i].set_title(f'Pressure Field (t={t})')

plt.tight_layout()
plt.show()


In [ ]:
traj_idx = 0
vel_mag = np.linalg.norm(vel[traj_idx, :, :, :, :], axis=-1)

# reduce dim with size 1 to 0
vel_mag = np.squeeze(vel_mag)

# transpose x and y
vel_mag = np.transpose(vel_mag, (0,  2, 1))

# Create normalized colormap for consistent color scaling
norm = Normalize(vmin=vel_mag.min(), vmax=vel_mag.max())

# Create frames
frames = []
for t in range(vel_mag.shape[0]):
    # Create figure and plot
    fig, ax = plt.subplots()
    im = ax.imshow(vel_mag[t,:,:])#, norm=norm)
    plt.colorbar(im)
    ax.set_title(f'Time step {t}')
    
    # Convert plot to image array
    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (4,))[:,:,:3]
    frames.append(frame)
    plt.close()

# Save as GIF
output_path = data_path.parent
output_path = output_path / (str(data_path.stem) + "_vel.gif")
print(f"Writing {len(frames)} frames to {output_path}")
iio.imwrite(output_path, frames, fps=30)
print(f"Animation saved to {output_path}")